In [1]:
!pip install pandas
!pip install jupyter_ui_poll
import time
from IPython.display import display, Image, clear_output, HTML
import random
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os



event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

image_path = {'_10v11_':'10v11.png', '_10v9_':'10v9.png', '_11v10_':'11v10.png', 
              '_11v12_':'11v12.png', '_12v11_':'12v11.png', '_12v14_':'12v14.png', 
              '_12v15_':'12v15.png', '_12v16_':'12v16.png', '_12v9_':'12v9.png', 
              '_13v14_':'13v14.png', '_14v12_':'14v12.png', '_14v13_':'14v13.png', 
              '_15v12_':'15v12.png', 
              '_15v16_':'15v16.png', '_15v18_':'15v18.png', '_15v20_':'15v20.png', 
              '_16v12_':'16v12.png', '_16v15_':'16v15.png', '_16v18_':'16v18.png', 
              '_16v20_':'16v20.png', '_17v18_':'17v18.png', '_18v15_':'18v15.png', 
              '_18v16_':'18v16.png', '_18v17_':'18v17.png', '_18v20_':'18v20.png', 
              '_19v20_':'19v20.png', '_20v15_':'20v15.png', '_20v16_':'20v16.png', 
              '_20v18_':'20v18.png', '_20v19_':'20v19.png', '_20v21_':'20v21.png', 
              '_21v18_':'21v18.png', '_21v20_':'21v20.png', '_9v10_':'9v10.png', 
              '_9v12_':'9v12.png',}



image_list = ['_10v11_', '_10v9_', '_11v10_',
              '_11v12_', '_12v11_', '_12v14_',
              '_12v15_', '_12v16_', 
              '_12v9_', '_13v14_',
              '_14v12_', '_14v13_', '_15v12_', 
              '_15v16_', '_15v18_', '_15v20_', 
              '_16v12_', '_16v15_', '_16v18_', 
              '_16v20_', '_17v18_', '_18v15_', 
              '_18v16_', '_18v17_', '_18v20_',
              '_19v20_', '_20v15_', '_20v16_', 
              '_20v18_', '_20v19_', '_20v21_',
              '_21v18_', '_21v20_', '_9v10_', 
              '_9v12_',]


random_image_list = image_list.copy()


random.shuffle(random_image_list)

def run_ans(image):
 
    score = 0
    
    
    print("Which side has more dots, left(l) or right(r)?")
    time.sleep(2)
    start_time = time.time()

    display(Image("blank.png", width=600))
    time.sleep(1)
    clear_output(wait = True)
    
    display(Image(image_path[image], width=600))
    time.sleep(0.75)
    clear_output(wait = False)

    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")

    btn1.on_click(register_event) 
    btn2.on_click(register_event)

    panel = widgets.HBox([btn1, btn2])
    display(panel)

    result = wait_for_event(timeout=3)
    clear_output()

    correct_ans = {'_10v11_':'Right', '_10v9_':'Left', '_11v10_':'Left', 
                   '_11v12_':'Right', '_12v11_':'Left', '_12v14_':'Right',
                   '_12v15_':'Right', '_12v16_':'Right', '_12v9_':'Left', 
                   '_13v14_':'Right', '_14v12_':'Left', '_14v13_':'Left', 
                   '_15v12_':'Left', '_15v16_':'Right',  '_15v18_':'Right', 
                   '_15v20_':'Right', '_16v12_':'Left', '_16v15_':'Left', 
                   '_16v18_':'Right', '_16v20_':'Right', '_17v18_':'Right', 
                   '_18v15_':'Left', '_18v16_':'Left', '_18v17_':'Left', 
                   '_18v20_':'Right', '_19v20_':'Right', '_18v9_':'Left', 
                   '_20v10_':'Left', '_20v15_':'Left', '_20v16_':'Left', 
                   '_20v18_':'Left', '_20v19_':'Left', '_20v21_':'Right', 
                   '_21v18_':'Left', '_21v20_':'Left', '_9v10_':'Right', 
                   '_9v12_':'Right',}

    if result['description'] == correct_ans[image]:
        score += 1
    
    else:
        score += 0

    
 
    end_time = time.time()
    
    time_taken = end_time - start_time
    
    
    print("score:", score)
    print ("time taken:",time_taken)
    time.sleep(1)
    clear_output(wait = False)
    
    return time_taken, score

def run_ans_set (random_image_list):

    results_dict = {'filename':[], 'time':[], 'score':[]}

    data_consent_info = """DATA CONSENT INFORMATION:

    Please read:

    we wish to record your response data

    to an anonymised public data repository.

    Your data will be used for educational teaching purposes

    practising data analysis and visualisation.

    Please type yes in the box below if you consent to the upload."""

    print(data_consent_info)
    result = input("> ")
    if result == "yes":

        print("Thanks for your participation.")

        print("Please contact philip.lewis@ucl.ac.uk")

        print("If you have any questions or concerns")

        print("regarding the stored results.")

    else:

        raise(Exception("User did not consent to continue test."))

    
    
    print("Welcome to the Approximate Number System test")
    time.sleep(3)
    
    id_instructions = """

    Enter your anonymised ID

    To generate an anonymous 4-letter unique user identifier please enter:

    - two letters based on the initials (first and last name) of a childhood friend

    - two letters based on the initials (first and last name) of a favourite actor / actress

    e.g. if your friend was called Charlie Brown and film star was Tom Cruise

    then your unique identifer would be CBTC

    """

    print(id_instructions)
    user_id = input("> ")

    print("User entered id:", user_id)
    

    print("Please enter your age.")
    age = input("> ")

    print("Please enter your gender.(Female/Male)")
    gender = input("> ")
    
    total_time = 0 
    total_score = 0


    
    random.shuffle(random_image_list)
    
    n = len(random_image_list)
    for i in range(n):
        image = random_image_list[i]
        time_taken, score = run_ans(image)
        filename = os.path.basename(image_path[image])
        results_dict['filename'].append(filename)
        results_dict['time'].append(time_taken)
        results_dict['score'].append(score)

        results_df = pd.DataFrame(results_dict)
        results_json = results_df.to_json()

        total_time += time_taken
        total_score += score
        time.sleep(0.75)
        clear_output(wait = False)
        
    print("total time taken:",total_time)
    print("total score:", total_score)


    results_df = pd.DataFrame(results_dict)
    results_json = results_df.to_json()       
    data_dict = {'user_id':user_id, 'age':age, 'gender':gender, 
                'results_json':results_json, 'total time':total_time,
                'total score': total_score}
    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfzOA6aOo8Epw6fEcQ7R19t9ckBBQBS1EvAechD0livDNM3ew/viewform?usp=sf_link'



  
    send_to_google_form(data_dict,form_url)
    
    
    
    return total_time, total_score

run_ans_set(random_image_list)
    

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
DATA CONSENT INFORMATION:

    Please read:

    we wish to record your response data

    to an anonymised public data repository.

    Your data will be used for educational teaching purposes

    practising data analysis and visualisation.

    Please type yes in the box below if you consent to the upload.


KeyboardInterrupt: Interrupted by user